# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,flin flon,54.7682,-101.8650,6.54,86,80,2.24,CA,1696486057
1,1,ilulissat,69.2167,-51.1000,-1.99,74,100,0.00,GL,1696486057
2,2,wailua homesteads,22.0669,-159.3780,25.24,90,100,2.06,US,1696486058
3,3,east london,-33.0153,27.9116,23.89,64,89,3.41,ZA,1696486058
4,4,rio grande,-32.0350,-52.0986,12.75,84,78,4.51,BR,1696486058


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) \
    & (city_data_df['Wind Speed'] < 4.5) \
    & (city_data_df['Cloudiness'] ==0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
70,70,orland,39.7474,-122.1964,24.17,49,0,3.23,US,1696486074
126,126,north bay,46.3168,-79.4663,21.17,76,0,4.12,CA,1696485960
218,218,gilgit,35.9221,74.3087,21.69,27,0,1.22,PK,1696486109
285,285,newman,37.3138,-121.0208,22.13,43,0,1.55,US,1696486124
302,302,hamilton,39.1834,-84.5333,21.10,77,0,2.57,US,1696485892
314,314,pleasanton,37.6624,-121.8747,23.42,58,0,0.00,US,1696485864
321,321,tobruk,32.0836,23.9764,22.62,65,0,2.27,LY,1696486132
349,349,vail,32.0479,-110.7120,21.80,26,0,0.89,US,1696486138
353,353,remire-montjoly,4.9167,-52.2667,25.02,94,0,0.00,GF,1696486139
376,376,laguna,38.4210,-121.4238,23.77,55,0,3.09,US,1696486144


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
70,orland,US,39.7474,-122.1964,49,
126,north bay,CA,46.3168,-79.4663,76,
218,gilgit,PK,35.9221,74.3087,27,
285,newman,US,37.3138,-121.0208,43,
302,hamilton,US,39.1834,-84.5333,77,
314,pleasanton,US,37.6624,-121.8747,58,
321,tobruk,LY,32.0836,23.9764,65,
349,vail,US,32.0479,-110.7120,26,
353,remire-montjoly,GF,4.9167,-52.2667,94,
376,laguna,US,38.4210,-121.4238,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
orland - nearest hotel: No hotel found
north bay - nearest hotel: Days Inn
gilgit - nearest hotel: PTDC Motel
newman - nearest hotel: No hotel found
hamilton - nearest hotel: North Vista Manor
pleasanton - nearest hotel: The Rose Hotel
tobruk - nearest hotel: فندق زهرة البطنان -طبرق
vail - nearest hotel: Hampton Inn & Suites Tucson Tech Park
remire-montjoly - nearest hotel: Complexe Belova
laguna - nearest hotel: Holiday Inn Express & Suites
sao jose de piranhas - nearest hotel: Hotel Pousada São José
agios nikolaos - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
70,orland,US,39.7474,-122.1964,49,No hotel found
126,north bay,CA,46.3168,-79.4663,76,Days Inn
218,gilgit,PK,35.9221,74.3087,27,PTDC Motel
285,newman,US,37.3138,-121.0208,43,No hotel found
302,hamilton,US,39.1834,-84.5333,77,North Vista Manor
314,pleasanton,US,37.6624,-121.8747,58,The Rose Hotel
321,tobruk,LY,32.0836,23.9764,65,فندق زهرة البطنان -طبرق
349,vail,US,32.0479,-110.7120,26,Hampton Inn & Suites Tucson Tech Park
353,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
376,laguna,US,38.4210,-121.4238,55,Holiday Inn Express & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)